<a href="https://colab.research.google.com/github/dennis-ancajima/ML-PUCP/blob/main/04_Modelo_Entrenamiento_12_atributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En el presente cuaderno vamos a realizar la experimentación de resultados propios considerando solo 12 características del conjunto de datos comparando el método de codificación Label Encoder vs Dummies aplicando diferentes métodos de clasificación para estimar los niveles de obesidad con un conjunto de datos  pre-procesado en el cuaderno: **03_Preprocesamiento.ipynb**. Y utilizando validación cruzada considerando (k=10)

 **Tabla de contenido:**

1. Cargar datos.
2. Modelado.

  2.2. Considerando la columna calculada: IMC y codificador: LabelEncoder.

  2.3. Sin considerar la columna calculada: IMC y codificador: dummies.

  2.4. Sin considerar la columna calculada: IMC y codificador: LabelEncoder.

4. Evaluación de Resultados



# 1. Cargar datos

En primero lugar, realizamos la importación de librerías y módulos necesarios para el análisis de datos y aprendizaje automático.

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt #visualizaciones
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score, roc_curve, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
import seaborn as sns #Libreria para gráficar
from sklearn.model_selection import cross_val_score
from google.colab import drive

pd.set_option("display.max_columns", 50) #Controla la cantidad máxima de columnas que se mostrarán al imprimir un DataFrame.

%matplotlib inline
plt.style.use('ggplot')

Realizamos la lectura de los archivo CSV pre-procesados sobre el conjunto de datos "ObesityDataSet_raw_and_data_sinthetic.csv" y los almacenamos en la siguientes variables utilizando la función read_csv() de la biblioteca panda.

In [9]:
#Lectura de los dataset pre-procesados
archivo1 = "12c_dataset_imc_dummies.csv"
archivo2 = "12c_dataset_imc_labelenconder.csv"
archivo3 = "12c_dataset_sinimc_dummies.csv"
archivo4 = "12c_dataset_sinimc_labelenconder.csv"

#El método "read_csv" nos permite leer un excel
dataset_imc_dummies = pd.read_csv(archivo1)
dataset_imc_labelenconder = pd.read_csv(archivo2)
dataset_sinimc_dummies = pd.read_csv(archivo3)
dataset_sinimc_labelenconder = pd.read_csv(archivo4)

# 2. Modelado

## 2.2. Considerando la columna calculada: IMC y codificador: LabelEncoder.

Veamos las primeras 5 filas del conjunto de datos almacenado en la variable dataset.

In [11]:
dataset_imc_labelenconder.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FCVC,NCP,CAEC,CH2O,FAF,TUE,CALC,NObeyesdad,IMC
0,0,21.0,1.62,64.0,1,2.0,3.0,2,2.0,0.0,1.0,3,1,24.3865
1,0,21.0,1.52,56.0,1,3.0,3.0,2,3.0,3.0,0.0,2,1,24.2382
2,1,23.0,1.80,77.0,1,2.0,3.0,2,2.0,2.0,1.0,1,1,23.7654
3,1,27.0,1.80,87.0,0,3.0,3.0,2,2.0,2.0,0.0,1,5,26.8519
4,1,22.0,1.78,89.8,0,2.0,1.0,2,2.0,0.0,0.0,2,6,28.3424


**División de datos de entrenamiento y test**

In [29]:
#Separamos la data, en variables independientes (x) y dependientes (y), para poder entrenar el modelo de regresión
X_train_val = dataset_imc_labelenconder.drop(['NObeyesdad'], axis=1)
y_train_val = dataset_imc_labelenconder['NObeyesdad']

Debido al tamaño del conjunto de datos se decidió utilizar el 70\% de los datos para el entrenamiento y el 30\% para la validación.

In [30]:
# Dividimos la data en conjunto de "entrenamiento" y "prueba"
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=0)

In [31]:
#Observamos el tamaño actual de los datos de entrenamiento y de validación.
print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_val.shape}")

Train shape: (1460, 13)
Test shape: (627, 13)


Decidimos utilizar el método **StandardScaler** para escalar los datos: siendo la media igual a 0 y la varianza igual a 1.

In [32]:
# Se crea un objeto de la clase StandardScaler de la librería scikit-learn.
scaler = StandardScaler()

X_train_scal = scaler.fit_transform(X_train) #Escalar los datos de entrenamiento
X_val_scal = scaler.fit_transform(X_val) #Escalar los datos de prueba

### Modelo: SVM Cúbico

In [33]:
# Tomado de notebook trabajado en clase "SVM - Ejercicio.ipynb"
from sklearn import svm
#Importa la función accuracy_score del módulo sklearn.metrics, que se utilizará para calcular la precisión del modelo.
from sklearn.metrics import accuracy_score

# Crea una instancia del clasificador SVM utilizando el kernel cúbico.
model = svm.SVC(kernel='poly', degree=3,C=0.0316, gamma=1)

# Entrenamiento del clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#Realiza una validación cruzada de 10 divisiones utilizando el modelo SVM y los datos de entrenamiento escalados.
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)

#Calcula la media de las puntuaciones de rendimiento obtenidas en la validación cruzada.
cvForest = cv.mean()
# Calcula la desviación estándar de las puntuaciones de rendimiento obtenidas en la validación cruzada.
stdForest = cv.std()


print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de SVM Cúbico: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.9006849315068493
Varianza de SVM Cúbico:  0.010273972602739719
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.9123


Obtenemos una exactitud de **0.9123**

### Modelo: Random Forest con bagging

In [34]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Crear una instancia del clasificador RandomForest
model = RandomForestClassifier(oob_score = True, random_state=42)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10) #metodo para obtener la precisión
cvForest = cv.mean()
stdForest = cv.std()

print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.986986301369863
Varianza de Random Forest:  0.009901939927945854
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.9601


### Modelo: Redes neuronales

In [35]:
#Importación de librerias
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Crear una instancia del clasificador MLPClassifier
model = MLPClassifier(random_state=1, max_iter=300)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#metodo para obtener la precisión
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)
cvForest = cv.mean()
stdForest = cv.std()

#Resultados
print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

promedio Validación cruzada k=10:  0.9397260273972602
Varianza de Random Forest:  0.013281314677852953
Exactitud en el conjunto de entrenamiento: 0.9993
Exactitud en el conjunto de validación: 0.9522


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


### Ensamble por votación manual

In [36]:
#Importación de bibliotecas y clasificadores
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm

# Crear  instancias para todos los siguientes clasificadores
estimators = [
     ('Random Forest', make_pipeline(StandardScaler(), RandomForestClassifier(oob_score = True, random_state=42))),
     ('SVM Cubic', make_pipeline(StandardScaler(),svm.SVC(kernel='poly', degree=3,C=10, gamma=50,probability=True))),

     ('árbol de decisión', DecisionTreeClassifier(max_depth=4)),
     ('k-NN', make_pipeline(StandardScaler(),
                            KNeighborsClassifier(n_neighbors=7) )),
     ('SVM de kernel gaussiano', make_pipeline(StandardScaler(),
                                               SVC(kernel='rbf', probability=True) ))
]

#Definición del ensamble
ensamble = VotingClassifier(estimators=estimators,
                        voting='soft', weights=[3,2,2, 1, 2])

#Ajuste del ensamble
ensamble.fit(X_train, y_train)

for (name, estimator) in ensamble.named_estimators_.items():
  print(f'Exactitud de {name: <25}: Entrenamiento {estimator.score(X_train, y_train):.03f} || Pruebas {estimator.score(X_val, y_val):.03f}',  )

print(f'Exactitud de {"ensamble": <25}: Entrenamiento {ensamble.score(X_train, y_train):.03f} || Validación {ensamble.score(X_val, y_val):.03f}' )

Exactitud de Random Forest            : Entrenamiento 1.000 || Pruebas 0.990
Exactitud de SVM Cubic                : Entrenamiento 1.000 || Pruebas 0.917
Exactitud de árbol de decisión        : Entrenamiento 0.977 || Pruebas 0.976
Exactitud de k-NN                     : Entrenamiento 0.866 || Pruebas 0.823
Exactitud de SVM de kernel gaussiano  : Entrenamiento 0.962 || Pruebas 0.927
Exactitud de ensamble                 : Entrenamiento 1.000 || Validación 0.984


**Conclusión**: Mejor modelo con Ensamble por votación manual, con una exactitud en el conjunto de validación de **0.984**

## 2.3. Sin considerar la columna calculada: IMC y codificador: dummies.

Veamos las primeras 5 filas del conjunto de datos almacenado en la variable dataset.

In [12]:
dataset_sinimc_dummies.head()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,NObeyesdad,EsMujer,History_Family,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no
0,21.0,1.62,64.0,2.0,3.0,2.0,0.0,1.0,Normal_Weight,True,True,0,0,1,0,0,0,0,1
1,21.0,1.52,56.0,3.0,3.0,3.0,3.0,0.0,Normal_Weight,True,True,0,0,1,0,0,0,1,0
2,23.0,1.80,77.0,2.0,3.0,2.0,2.0,1.0,Normal_Weight,False,True,0,0,1,0,0,1,0,0
3,27.0,1.80,87.0,3.0,3.0,2.0,2.0,0.0,Overweight_Level_I,False,False,0,0,1,0,0,1,0,0
4,22.0,1.78,89.8,2.0,1.0,2.0,0.0,0.0,Overweight_Level_II,False,False,0,0,1,0,0,0,1,0


**División de datos de entrenamiento y test**

In [13]:
#Separamos la data, en variables independientes (x) y dependientes (y), para poder entrenar el modelo de regresión
X_train_val = dataset_sinimc_dummies.drop(['NObeyesdad'], axis=1)
y_train_val = dataset_sinimc_dummies['NObeyesdad']

Debido al tamaño del conjunto de datos se decidió utilizar el 70\% de los datos para el entrenamiento y el 30\% para la validación.

In [14]:
# Dividimos la data en conjunto de "entrenamiento" y "prueba"
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=0)

In [15]:
#Observamos el tamaño actual de los datos de entrenamiento y de validación.
print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_val.shape}")

Train shape: (1460, 18)
Test shape: (627, 18)


Decidimos utilizar el método **StandardScaler** para escalar los datos: siendo la media igual a 0 y la varianza igual a 1.

In [16]:
# Se crea un objeto de la clase StandardScaler de la librería scikit-learn.
scaler = StandardScaler()

X_train_scal = scaler.fit_transform(X_train) #Escalar los datos de entrenamiento
X_val_scal = scaler.fit_transform(X_val) #Escalar los datos de prueba

### Modelo: SVM Cúbico

In [17]:
# Tomado de notebook trabajado en clase "SVM - Ejercicio.ipynb"
from sklearn import svm
#Importa la función accuracy_score del módulo sklearn.metrics, que se utilizará para calcular la precisión del modelo.
from sklearn.metrics import accuracy_score

# Crea una instancia del clasificador SVM utilizando el kernel cúbico.
model = svm.SVC(kernel='poly', degree=3,C=0.0316, gamma=1)

# Entrenamiento del clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#Realiza una validación cruzada de 10 divisiones utilizando el modelo SVM y los datos de entrenamiento escalados.
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)

#Calcula la media de las puntuaciones de rendimiento obtenidas en la validación cruzada.
cvForest = cv.mean()
# Calcula la desviación estándar de las puntuaciones de rendimiento obtenidas en la validación cruzada.
stdForest = cv.std()


print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de SVM Cúbico: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.8678082191780823
Varianza de SVM Cúbico:  0.02526831061054344
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.8884


Observamos un síntoma claro de sobreajuste (overfitting).

### Modelo: Random Forest con bagging

In [18]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Crear una instancia del clasificador RandomForest
model = RandomForestClassifier(oob_score = True, random_state=42)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10) #metodo para obtener la precisión
cvForest = cv.mean()
stdForest = cv.std()

print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.9465753424657534
Varianza de Random Forest:  0.020272121351984567
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.9234


Con Random Forest se obtiene una exactitud en el conjunto de validación de **0.9234**

### Modelo: Redes neuronales

In [19]:
#Importación de librerias
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Crear una instancia del clasificador MLPClassifier
model = MLPClassifier(random_state=1, max_iter=300)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#metodo para obtener la precisión
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)
cvForest = cv.mean()
stdForest = cv.std()

#Resultados
print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

promedio Validación cruzada k=10:  0.923972602739726
Varianza de Random Forest:  0.016902688601716533
Exactitud en el conjunto de entrenamiento: 0.9986
Exactitud en el conjunto de validación: 0.9219


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


**Conclusión**: Mejor modelo con Random Forest, con una exactitud en el conjunto de validación de **0.9234**

## 2.4. Sin considerar la columna calculada: IMC y codificador: LabelEncoder.

Veamos las primeras 5 filas del conjunto de datos almacenado en la variable dataset.

In [20]:
dataset_sinimc_labelenconder.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FCVC,NCP,CAEC,CH2O,FAF,TUE,CALC,NObeyesdad
0,0,21.0,1.62,64.0,1,2.0,3.0,2,2.0,0.0,1.0,3,1
1,0,21.0,1.52,56.0,1,3.0,3.0,2,3.0,3.0,0.0,2,1
2,1,23.0,1.80,77.0,1,2.0,3.0,2,2.0,2.0,1.0,1,1
3,1,27.0,1.80,87.0,0,3.0,3.0,2,2.0,2.0,0.0,1,5
4,1,22.0,1.78,89.8,0,2.0,1.0,2,2.0,0.0,0.0,2,6


**División de datos de entrenamiento y test**

In [21]:
#Separamos la data, en variables independientes (x) y dependientes (y), para poder entrenar el modelo de regresión
X_train_val = dataset_sinimc_labelenconder.drop(['NObeyesdad'], axis=1)
y_train_val = dataset_sinimc_labelenconder['NObeyesdad']

Debido al tamaño del conjunto de datos se decidió utilizar el 70\% de los datos para el entrenamiento y el 30\% para la validación.

In [22]:
# Dividimos la data en conjunto de "entrenamiento" y "prueba"
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=0)

In [23]:
#Observamos el tamaño actual de los datos de entrenamiento y de validación.
print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_val.shape}")

Train shape: (1460, 12)
Test shape: (627, 12)


Decidimos utilizar el método **StandardScaler** para escalar los datos: siendo la media igual a 0 y la varianza igual a 1.

In [24]:
# Se crea un objeto de la clase StandardScaler de la librería scikit-learn.
scaler = StandardScaler()

X_train_scal = scaler.fit_transform(X_train) #Escalar los datos de entrenamiento
X_val_scal = scaler.fit_transform(X_val) #Escalar los datos de prueba

### Modelo: SVM Cúbico

In [25]:
# Tomado de notebook trabajado en clase "SVM - Ejercicio.ipynb"
from sklearn import svm
#Importa la función accuracy_score del módulo sklearn.metrics, que se utilizará para calcular la precisión del modelo.
from sklearn.metrics import accuracy_score

# Crea una instancia del clasificador SVM utilizando el kernel cúbico.
model = svm.SVC(kernel='poly', degree=3,C=0.0316, gamma=1)

# Entrenamiento del clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#Realiza una validación cruzada de 10 divisiones utilizando el modelo SVM y los datos de entrenamiento escalados.
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)

#Calcula la media de las puntuaciones de rendimiento obtenidas en la validación cruzada.
cvForest = cv.mean()
# Calcula la desviación estándar de las puntuaciones de rendimiento obtenidas en la validación cruzada.
stdForest = cv.std()


print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de SVM Cúbico: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.8753424657534247
Varianza de SVM Cúbico:  0.012555001903988604
Exactitud en el conjunto de entrenamiento: 0.9986
Exactitud en el conjunto de validación: 0.8995


Obtenemos una exactitud de **0.8995** (Overfitting)

### Modelo: Random Forest con bagging

In [26]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Crear una instancia del clasificador RandomForest
model = RandomForestClassifier(oob_score = True, random_state=42)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10) #metodo para obtener la precisión
cvForest = cv.mean()
stdForest = cv.std()

print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.9417808219178083
Varianza de Random Forest:  0.022561190536987148
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.9250


### Modelo: Redes neuronales

In [27]:
#Importación de librerias
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Crear una instancia del clasificador MLPClassifier
model = MLPClassifier(random_state=1, max_iter=300)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#metodo para obtener la precisión
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)
cvForest = cv.mean()
stdForest = cv.std()

#Resultados
print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

promedio Validación cruzada k=10:  0.9294520547945206
Varianza de Random Forest:  0.015931100478921923
Exactitud en el conjunto de entrenamiento: 0.9979
Exactitud en el conjunto de validación: 0.9426


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


### Ensamble por votación manual

In [28]:
#Importación de bibliotecas y clasificadores
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm

# Crear  instancias para todos los siguientes clasificadores
estimators = [
     ('Random Forest', make_pipeline(StandardScaler(), RandomForestClassifier(oob_score = True, random_state=42))),
     ('SVM Cubic', make_pipeline(StandardScaler(),svm.SVC(kernel='poly', degree=3,C=10, gamma=50,probability=True))),

     ('árbol de decisión', DecisionTreeClassifier(max_depth=4)),
     ('k-NN', make_pipeline(StandardScaler(),
                            KNeighborsClassifier(n_neighbors=7) )),
     ('SVM de kernel gaussiano', make_pipeline(StandardScaler(),
                                               SVC(kernel='rbf', probability=True) ))
]

#Definición del ensamble
ensamble = VotingClassifier(estimators=estimators,
                        voting='soft', weights=[3,2,2, 1, 2])

#Ajuste del ensamble
ensamble.fit(X_train, y_train)

for (name, estimator) in ensamble.named_estimators_.items():
  print(f'Exactitud de {name: <25}: Entrenamiento {estimator.score(X_train, y_train):.03f} || Pruebas {estimator.score(X_val, y_val):.03f}',  )

print(f'Exactitud de {"ensamble": <25}: Entrenamiento {ensamble.score(X_train, y_train):.03f} || Validación {ensamble.score(X_val, y_val):.03f}' )

Exactitud de Random Forest            : Entrenamiento 1.000 || Pruebas 0.952
Exactitud de SVM Cubic                : Entrenamiento 1.000 || Pruebas 0.895
Exactitud de árbol de decisión        : Entrenamiento 0.805 || Pruebas 0.788
Exactitud de k-NN                     : Entrenamiento 0.847 || Pruebas 0.793
Exactitud de SVM de kernel gaussiano  : Entrenamiento 0.941 || Pruebas 0.884
Exactitud de ensamble                 : Entrenamiento 0.996 || Validación 0.944


**Conclusión**: Mejor modelo con Ensamble por votación manual, con una exactitud en el conjunto de validación de **0.944**

# 3. Evaluación de Resultados

Se concluye que el mejor modelo tuvo una exactitud de **0.984** con el Ensamble por votación manual, con una validación cruzada de 10 folders.